In [1]:
!pip install openai

In [2]:
## Get API from 'apikey' file
filename = '../apikey'
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)
api_key = get_file_contents(filename)
# print("Your API key is: %s" % (api_key))

In [3]:
from openai import OpenAI

client = OpenAI(
  api_key= api_key
)

In [4]:
## Testing ChatGPT API
completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": "You are a poetic assistant, skilled in explaining complex programming concepts with creative flair."},
    {"role": "user", "content": "Compose a poem that explains the concept of recursion in programming."}
  ]
)

print(completion.choices[0].message)

ChatCompletionMessage(content="In the realm of code, a wondrous delight,\nThere lies a concept, both clever and bright,\nRecursion, a loop elegant and profound,\nWhere self-referential magic can be found.\n\nPicture a function, humble and small,\nWith the power to ignite, the ability to enthral,\nIt calls itself, a mirror of its own grace,\nUnfolding its essence, in an enchanting embrace.\n\nLike a pebble dropped in a tranquil lake,\nReverberations of recursion, a path it will make,\nExploring the depths with each passing theme,\nUnraveling infinite loops, all part of its scheme.\n\nWith each iteration, a world anew,\nA dance of procedure, as patterns accrue,\nProblems untangle, their solutions unfold,\nAs recursion seeks truth, steadfast and bold.\n\nIn the labyrinth of numbers, it finds its zest,\nDividing and conquering, it's always the best,\nSplitting a problem, into fragments so small,\nRecursive brilliance, conquers them all.\n\nA Fibonacci sequence, a recursive affair,\nWhere n

In [5]:
def summarize_dataset(df, df_name, category):

  col = df.columns

  if category == "Market Dataset":

    system_prompt = "You are an experienced Market financial analyst that understand all market dataset, including Stock Price market, Commodity Price market and Currecy Exchange Price market."
  

  elif category == "Economic Dataset":

    system_prompt = "You are an experienced Economist that understand all economic dataset, including Gross Domestic Product, Unemployment Rate and Consumer Price Index"


  elif category == "Corporate Dataset":

    system_prompt = " You are an experienced Corporate financial analyst that understand all company statement, including Income Statement and Cash Flow Statement."
   
  else:
    return None

  system_prompt += """
  You can figure out the TOP3 metrics derived from the dataset.
  TOP means each data row can calculate a new metric.
  ONLY return the result in a form of "["name 1", "name 2", "name 3"]"
  """

  message = f"""
  I have a dataset called {df_name}, with its columns names below
  {list(col)}
  What are the TOP3 metrics I can derived from this dataset? (without explaination) 
  """

  completion = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[
      {"role": "system", "content": system_prompt},
      {"role": "assistant", "content": message},
    ]
  )

  return completion.choices[0].message.content

In [6]:
def create_new_col(new_cols, df, df_name, category):

  col = df.columns

  if category == "Market Dataset":

    system_prompt = """
    You are an experienced Market financial analyst that understand all market dataset, including Stock Price market, Commodity Price market and Currecy Exchange Price market.
    You know all the calculation of financial concpets.
    """

  elif category == "Economic Dataset":

    system_prompt = """
    You are an experienced Economist that understand all economic dataset, including Gross Domestic Product, Unemployment Rate and Consumer Price Index.
    You know all the calculation of economic concpets.
    """

  elif category == "Corporate Dataset":
    
    system_prompt = """
    You are an experienced Economist that understand all economic dataset, including Gross Domestic Product, Unemployment Rate and Consumer Price Index.
    You know all the calculation of economic concpets.
    """

  else:
    return None


  system_prompt += """
  i) ONLY return Executable Python code
  ii) NO NEED TO read the dataset into a dataframe
  iii) DO NOT print out the new values
  iv) USE "df" to represent the dataframe in the code
  """

  message = f"""
  I have a dataset called {df_name}, with its columns names below
  {list(col)}
  
  Can you provide executable Python code to add the following columns
  {new_cols}

  ONLY return Executable Python code
  USE "df" to represent the dataframe in the code
  Code must not require further input
  
  """

  completion = client.chat.completions.create(
  model="gpt-3.5-turbo",
  messages=[
    {"role": "system", "content": system_prompt},
    {"role": "assistant", "content": message},
  ])

  return completion.choices[0].message.content



In [7]:
import pandas as pd
import json
import time

Stock_price = pd.read_csv("../example/data/Stock_price_TSLA.csv")

new_metrics = summarize_dataset(Stock_price, "Stock_price_TSLA", "Market Dataset")

new_metrics = json.loads(new_metrics)

print(new_metrics)

print(create_new_col(new_metrics, Stock_price, "Stock_price_TSLA", "Market Dataset"))

time.sleep(60)

/Users/borisyu/opt/anaconda3/lib/python3.8/site-packages/pandas/core/computation/expressions.py:20: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.2' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


['Daily price range', 'Average daily price', 'Volume-weighted average price']
Sure, here is the Python code to add the requested columns to the dataframe:

```python
df['Daily price range'] = df['High'] - df['Low']
df['Average daily price'] = (df['Open'] + df['Close']) / 2
df['Volume-weighted average price'] = (df['Close'] * df['Volume']).cumsum() / df['Volume'].cumsum()
```

Please note that this code assumes that the dataframe `df` already contains the columns mentioned in the question (`['Date', 'Open', 'High', 'Low', 'Close', 'Adj Close', 'Volume']`).


In [8]:
CPI = pd.read_csv("../example/data/Consumer_price_index.csv")

new_metrics = summarize_dataset(CPI, "Consumer_price_index", "Economic Dataset")

new_metrics = json.loads(new_metrics)

print(new_metrics)

print(create_new_col(new_metrics, CPI, "Consumer_price_index", "Economic Dataset"))

time.sleep(60)

['Inflation rate', 'Core inflation rate', 'Energy inflation rate']
To calculate the inflation rate, we can use the percentage change formula:

Inflation rate = ((current value - previous value) / previous value) * 100

To calculate the core inflation rate, we can calculate the percentage change in the "All-items excluding food and energy 7" column.

To calculate the energy inflation rate, we can calculate the percentage change in the "Energy 7" column.

Here's the executable Python code to add the new columns:

```python
# Calculate the inflation rate
df['Inflation rate'] = ((df['All-items'] - df['All-items'].shift(1)) / df['All-items'].shift(1)) * 100

# Calculate the core inflation rate
df['Core inflation rate'] = ((df['All-items excluding food and energy 7'] - df['All-items excluding food and energy 7'].shift(1)) / df['All-items excluding food and energy 7'].shift(1)) * 100

# Calculate the energy inflation rate
df['Energy inflation rate'] = ((df['Energy 7'] - df['Energy 7'].shift(1

In [9]:
income_statement = pd.read_csv("../example/data/Income_statement.csv")

new_metrics = summarize_dataset(income_statement, "Income_statement", "Corporate Dataset")

new_metrics = json.loads(new_metrics)

print(new_metrics)

print(create_new_col(new_metrics, income_statement, "Income_statement", "Corporate Dataset"))

['Gross Profit Margin', 'Operating Margin', 'Net Profit Margin']
df['Gross Profit Margin'] = df['Gross Profit'] / df['Total Revenue'] * 100
df['Operating Margin'] = df['Operating Income/Expenses'] / df['Total Revenue'] * 100
df['Net Profit Margin'] = df['Net Income Available to Common Stockholders'] / df['Total Revenue'] * 100
